In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from pymongo import MongoClient
import urllib.parse
import matplotlib.pyplot as plt
from PIL import Image

import os
import sys
import json
from json import dumps

# From within the docker-compose application, use "mongodb_container", for local development use "localhost"
dbServer = os.getenv("MONGO_DB_SERVER", "mongodb_container:27017")
dbUser = os.getenv("MONGO_USERNAME", "root")
dbPW = os.getenv("MONGO_PASSWORD", "rootpassword")

client = MongoClient(dbServer, username=dbUser, password=dbPW)

print("Mongo DB Connection -----")
print("server:" + dbServer)
print("user:" + dbUser)

collection = client.images.images

In [ ]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm

def get_clip_vectors():
    data = []
    for image in (
        collection.find(
            {"clip":{"$exists": "true", "$not": {"$size": 0}}},
            {
                "clip": 1,
                "_id": 0,
                "filenameHash":1
            },
        )
    ):
        data.append(image)

    returnString = data
    return returnString

vectors=get_clip_vectors()

m=len(vectors)
# Clip uses a 512-dimensional vector
n=512
matrix=np.zeros((m,n),np.half)
filenames=[]
for i in range(len(vectors)):
    #print(i)
    matrix[i,:]=normalize(dict(vectors[i])['clip'][0])
    filenames.append(dict(vectors[i])['filenameHash'])
    
matrix.shape
#np.savez_compressed('test.npz',matrix)

In [ ]:
queryString="A group of bicycles"
#queryString="Rail"

import requests
metadataClipServer = os.getenv("METADATA_CLIP", "http://metadata_clip:8003/uploadtext/"+queryString)
clip_response = requests.get(metadataClipServer).json()
# Create Vector and normalize it
matrixB=np.zeros((n,1),np.half)
matrixB[:,0]=normalize(clip_response[0])
# Multiply with Images Matrix
result=np.matmul(matrix,matrixB)
# Sort the result
indizes=np.argsort(result[:,0])[::-1]
resultindex=list(indizes)
df=pd.DataFrame(filenames,columns=['filename'])
dfres=df.filename[resultindex].tail(10)
dfres.values[0]
# Show 10 best images
for i in range(2):
    a=Image.open('/home/jovyan/data/imageblobstorage/'+dfres.values[i]+'.png')
    a.show()